## 🏆&nbsp;&nbsp;Load in the Olympics Data


In [1]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.express as px

# Import the data
olympics_data = pd.read_csv("data/athlete_events.csv.gz")

# Preview the DataFrame
olympics_data

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


The `team` column is messy and sometimes contains countries separated by forward slashes or hyphens. Let's clean this by using [`.str.extract()`]

In [3]:
# Split the team column on hyphens and slashes
olympics_data["team_clean"] = olympics_data["team"].str.split(r"[\/\-]").str[0]

# Preview the new column
olympics_data["team_clean"].unique()[:10]

array(['China', 'Denmark', 'Netherlands', 'United States', 'Finland',
       'Norway', 'Romania', 'Estonia', 'France', 'Taifun'], dtype=object)

## 🌐&nbsp;&nbsp;Bring in additional data

I may find using additional world data helpful. 

In [2]:
-- Select country data based on most recent year
WITH world_countries AS (
    SELECT 
        countries.name AS country, 
        continents.name AS continent,  
		year,
    	population,
    	gdp,
		ROW_NUMBER() OVER(PARTITION BY country ORDER BY year DESC) AS year_index
	FROM countries
    INNER JOIN country_stats USING(country_id)
    INNER JOIN regions USING(region_id)
    INNER JOIN continents USING(continent_id)
    GROUP BY 1, 2, 3, 4, 5)

SELECT
	country,
    continent,
    year,
    population,
    gdp
FROM world_countries
WHERE year_index = 1

,country,continent,year,population,gdp
0,Afghanistan,Asia,2018,37172386,19362969582
1,Albania,Europe,2018,2866376,15058879129
2,Algeria,Africa,2018,42228429,180689000000
3,American Samoa,Oceania,2017,55620,634000000
4,Andorra,Europe,2018,77006,3236543909
...,...,...,...,...,...
198,Vietnam,Asia,2018,95540395,244948000000
199,"Virgin Islands, U.S.",North America,2017,107268,3855000000
200,Yemen,Asia,2018,28498687,26914402224
201,Zambia,Africa,2018,17351822,26720073436


We now have country data that we can combine with the Olympics data. We will use the `.merge()`.

A "left" join matches on rows in the olympics_data DataFrame, as some teams will not be present in the `countries_data` DataFrame. I then filter for rows where the sport is "Judo".

In [5]:
# Perform a left join between the two DataFrames using the country columns
olympics_country_data = olympics_data.merge(
    countries_data, left_on="team_clean", right_on="country", how="left"
)

# Filter for Judo
judo_data = olympics_country_data[olympics_country_data["sport"] == "Judo"]

# Preview the data
judo_data

,id,name,sex,age,height,weight,team,noc,games,year_x,season,city,sport,event,medal,team_clean,country,continent,year_y,population,gdp
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,China,Asia,2018.0,1.392730e+09,1.360820e+13
160,64,M'Bairo Abakar,M,31.0,NaN,NaN,Chad,CHA,1992 Summer,1992,Summer,Barcelona,Judo,Judo Men's Half-Middleweight,NaN,Chad,Chad,Africa,2018.0,1.547775e+07,1.130254e+10
180,78,Salim Abanoz,M,26.0,NaN,NaN,Turkey,TUR,1996 Summer,1996,Summer,Atlanta,Judo,Judo Men's Lightweight,NaN,Turkey,Turkey,Asia,2018.0,8.231972e+07,7.665090e+11
412,229,Mohamed Ali Abdelaal,M,26.0,175.0,81.0,Egypt,EGY,2016 Summer,2016,Summer,Rio de Janeiro,Judo,Judo Men's Half-Middleweight,NaN,Egypt,Egypt,Africa,2018.0,9.842360e+07,2.508950e+11
435,242,Ahmed Abdelrahman,M,20.0,165.0,60.0,Egypt,EGY,2016 Summer,2016,Summer,Rio de Janeiro,Judo,Judo Men's Extra-Lightweight,NaN,Egypt,Egypt,Africa,2018.0,9.842360e+07,2.508950e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270853,135459,Oleg Zurabiani,M,19.0,172.0,64.0,Soviet Union,URS,1976 Summer,1976,Summer,Montreal,Judo,Judo Men's Lightweight,NaN,Soviet Union,NaN,NaN,NaN,NaN,NaN
270918,135482,Goran uvela,M,27.0,184.0,93.0,Yugoslavia,YUG,1976 Summer,1976,Summer,Montreal,Judo,Judo Men's Half-Heavyweight,NaN,Yugoslavia,NaN,NaN,NaN,NaN,NaN
270981,135503,Zurab Zviadauri,M,23.0,182.0,90.0,Georgia,GEO,2004 Summer,2004,Summer,Athina,Judo,Judo Men's Middleweight,Gold,Georgia,Georgia,Asia,2018.0,3.731000e+06,1.620982e+10
271032,135535,Claudia Antoinette Zwiers,F,22.0,181.0,78.0,Netherlands,NED,1996 Summer,1996,Summer,Atlanta,Judo,Judo Women's Middleweight,Bronze,Netherlands,Netherlands,Europe,2018.0,1.723102e+07,9.136580e+11


## 🎨&nbsp;&nbsp;Visualize the data 

I will begin with a boxplot to identify differences in the medal counts between continents.

First, I will need to create a column named `medal_count`. This column will represent the number of non-null values by country (i.e., the number of medals). 

In [6]:
# Group by the team and calculate the number of medals earned
judo_data["medal_count"] = judo_data.groupby("team_clean")["medal"].transform(
    lambda x: x.notnull().sum()
)

# Create a boxplot of the medal count by continent
fig = px.box(judo_data, x="continent", y="medal_count", hover_data=["team_clean"], title="Medal count by country")
fig.show()

## 🔬&nbsp;&nbsp;Go forth and analyze!
Well done! You have successfully used Python and SQL to load data from multiple sources, process it, and begin to perform an analysis. It's now up to you to further explore the data, create new features, and estimate the probability that a judo athlete will earn a medal.

Not sure where to go from here? Here are a few potential next steps:
- Some countries lack country data because they have different names in the `olympics_data` and the `countries_data` (e.g., "Great Britain" and "United Kingdom"). Can you resolve these discrepancies?
- How do athlete statistics, such as height and weight, relate to judo performance?
- What type of model(s) will you use to predict whether an athlete wins a medal? Consider checking out [Machine Learning with scikit-learn](https://app.datacamp.com/learn/courses/machine-learning-with-scikit-learn) if you want a refresher on machine learning in Python!

When you're finished, be sure to share your analysis. Click the **Share** button to build a sleek publication that includes the rendered  text, code cells, and code output. When you publish your work, it appears automatically on your DataCamp profile so you can build out your portfolio. If you do not want to share your work with the world, you can adjust the sharing settings.